# Linked List

A linked list is a collection of values arranged in a linear, unidirectional sequence.
- Constant time insertion and removal from the front of the list.
- Reliable performance characteristics.

![Linked List](linkedlist.png)

A linked list is a chain of nodes.

## Nodes

Nodes have two responsibilities:
- Hold a value.
- Hold a reference to the next node. the absence of a reference to the next node, marks the end of the list
![Node](node.png)

In [1]:
data class Node<T: Any>(
    var value: T,
    var next: Node<T>? = null
) {
    override fun toString(): String {
        return if (next == null) "$value"
        else "$value -> ${next.toString()}"
    }
}

Note: Using `T: Any` to set an upper bound for the type parameter ensures that `T` will always be a non-nullable type.

Creating and linking nodes:

In [2]:
val node1 = Node(value = 1)
val node2 = Node(value = 2)
val node3 = Node(value = 3)
node1.next = node2
node2.next = node3

println(node1)

1 -> 2 -> 3


![Nodes Connected](nodes_connected.png)

## LinkedList

In [3]:
class LinkedList<T : Any> {
    // private
    var head: Node<T>? = null
    // private
    var tail: Node<T>? = null
    // private
    var size = 0

    fun isEmpty(): Boolean = size == 0

    override fun toString(): String {
        return if (isEmpty()) "EMPTY list"
        else head.toString()
    }
}

> Note: to can add functions to the class, we will be commenting private variables and functions.

![Linked ListParts](linkedList_parts.png)

## Adding values to the list

1. `push`: Adds values at the front of the list. **Head-first insertion**
2. `append` Adds a value at the end of the list.
3. `insert`: Adds a value after a particular node of the list.

In [4]:
fun <T : Any> LinkedList<T>.push(value: T) {
    head = Node(value = value, next = head)
    if (tail == null) tail = head
    size++
}

You create a new `Node` which holds the new value, and points to the node that was previously the `head` of the list.

In the case in which you're pushing into an empty list, the new node is both the `head` and `tail`of the list.

Example of push:

In [5]:
val list = LinkedList<Int>()
list.push(3)
list.push(2)
list.push(1)

println(list)

1 -> 2 -> 3


We'll use the **fluent interface** pattern to chain multiple `push`calls.

In [6]:
fun <T: Any> LinkedList<T>.push(value: T): LinkedList<T> = apply { 
    head = Node(value = value, next = head)
    if (null == tail) tail = head
    size++
 }

Example of fluent interface push

In [7]:
val list = LinkedList<Int>()
list.push(3).push(2).push(1)
println(list)

1 -> 2 -> 3


## Append operations

This adds a value at the end of the list, which is known as **tail-end insertion** 

In [8]:
fun <T: Any> LinkedList<T>.append(value: T) {
    // 1
    if (isEmpty()) {
        push(value)
        return
    }
    // 2
    val newNode = Node(value = value)
    tail?.next = newNode
    // 3
    tail = newNode
    size++
}

1. If the list is empty, you'll need to update both `head` and `tail`. Since `append` on an empty list is functionally identical to `push`, you invoke `push`to do the work for you.
2. In all other cases, you create a new node *after* the `tail` node.
3. Also, the new node is the tail of the list.

Example of append:

In [9]:
val list = LinkedList<Int>()
list.append(1)
list.append(2)
list.append(3)

println(list)

1 -> 2 -> 3


Append with fluent interface:

In [10]:
fun <T: Any> LinkedList<T>.append(value: T): LinkedList<T> = apply { 
    if (isEmpty()) {
        push(value)
        return@apply
    }
    val newNode = Node(value = value)
    tail?.next = newNode
    tail = newNode
    size++
 }

In [11]:
val list = LinkedList<Int>()
list.append(1).append(2).append(3)

println(list)

1 -> 2 -> 3


## Insert operations

This operation inserts a value at a particular place in the list and requires two steps:

1. Finding a particular node in the list.
2. Inserting the new node after that node.


In [12]:
fun <T: Any> LinkedList<T>.nodeAt(index: Int): Node<T>? {
    // 1
    var currentNode = head
    var currentIndex = 0
    
    // 2
    while (currentNode != null && currentIndex < index) {
        currentNode = currentNode.next
        currentIndex++
    }
    return currentNode
}

`nodeAt()` tries to retrieve a node in the list based on the given index. Since you can only access the nodes of the list from the head node, you'll have to make iterative traversals.

1. You create a new reference to `head`and keep track of the current number of steps taken in the list.
2. Using a `while` loop, you move the reference down the list until you reach the desired index.

Now you need to insert the new node.
 

In [13]:
fun <T: Any> LinkedList<T>.insert(value: T, afterNode: Node<T>): Node<T> {
    // 1
    if (tail == afterNode) {
        append(value)
        return tail!!
    }
    // 2
    val newNode = Node(value = value, next = afterNode.next)
    // 3
    afterNode.next = newNode
    size++
    return newNode
}

1. In the case where this method is called with the `tail`node, you call the functionally equivalent `append` method. This takes care of updating `tail`.
2. Otherwise, you create a new node and lint its `next` property to the next node of the specified Node.
3. You reassign the `next` value of the specified node to link  it to the new node that you just created.

Inserting at a particular index:

In [14]:
val list = LinkedList<Int>()
list.push(3).push(2).push(1)

println("Before inserting: $list")

var middleNode = list.nodeAt(1)!!
for (i in 1..3) {
    middleNode = list.insert(-1 * i, middleNode)
}
println("After inserting: $list")

Before inserting: 1 -> 2 -> 3
After inserting: 1 -> 2 -> -1 -> -2 -> -3 -> 3


## Performance analysis

![Linked List performance](linkedList_perfomance.png)

## Removing values from the list

1. `pop`: Removes the value at the front of the list.
2. `removeLast`: Removes the value at the end of the list.
3. `removeAfter`: Removes a value after a particular node of the list.

## Pop operations

In [15]:
fun <T: Any> LinkedList<T>.pop(): T? {
    if (isEmpty()) return null
    // 1
    val result = head?.value
    // 2
    head = head?.next
    size--
    // 3
    if (isEmpty()) tail = null
    return result
}

1. `pop()` returns the value that was removed.
2. By moving the `head` down a node, you've effectively removed the first node of the list. The garbage collector will remove the old node from memory once the method finishes since there will be no more references attached to it. 
3. If the list becomes empty, you set `tail` to `null` as well.

Example of pop:

In [16]:
val list = LinkedList<Int>()
list.push(3).push(2).push(1)
println("Before popping list $list")
val poppedValue = list.pop()
println("After popping list $list")
println("Popped value: $poppedValue")

Before popping list 1 -> 2 -> 3
After popping list 2 -> 3
Popped value: 1


## Remove last operations

Although you have a reference to the `tail` node, you can´t chop it off without having a reference to the node before it. Thus, you need to traverse the whole list to find the node before the last.

In [17]:
fun <T: Any> LinkedList<T>.removeLast(): T? {
    // 1
    val head = head ?: return null
    // 2
    if (head.next == null) return pop()
    // 3
    size--
    
    // 4
    var prev = head
    var current = head
    var next = current.next
    while (next != null) {
        prev = current
        current = next
        next = current.next
    }
    // 5
    prev.next = null
    tail = prev
    return current.value
}

1. If `head`is `null`, there's nothing to remove, so you return `null`.
2. If the list only consists of one node, `removeLast` is functionally equivalent to `pop`.
3. At this point, you know that you'll be removing a node, so you update the size of the list accordingly.
4. You keep searching fot the next node until `current.next`is `null`. This signifies that `current`is the last node of the list.
5. Since `current`is the last node, you disconnect it using the `prev.next`reference. You also make sure to update the `tail` reference.

Example of removing the last node:

In [18]:
val list = LinkedList<Int>().push(3).push(2).push(1)
println("Before removing last node: $list")

val removedValue = list.removeLast()

println("After removing last node: $list")
println("Removed value: $removedValue")

Before removing last node: 1 -> 2 -> 3
After removing last node: 1 -> 2
Removed value: 3


`removeLast()` is an *O(n)* operation, which is relatively expansive.

## Remove after operations

You'll first find the node immediately before the node you wish to remove and then unlink it.
![Removing the middle node](removing_after.png)

In [19]:
fun <T: Any> LinkedList<T>.removeAfter(node: Node<T>): T? {
    val result = node.next?.value
    
    if (node.next == tail) {
        tail = node
    }
    if (node.next != null) {
        size--
    }
    node.next = node.next?.next
    return result
}

Example of removing a node after a particular node:

In [20]:
val list = LinkedList<Int>().push(3).push(2).push(1)
println("Before removing at particular index: $list")

val indexToRemove = 1
val node = list.nodeAt(indexToRemove - 1)!!
val removedValue = list.removeAfter(node)
println("After removing at index $indexToRemove: $list")
println("Removed value: $removedValue")

Before removing at particular index: 1 -> 2 -> 3
After removing at index 1: 1 -> 3
Removed value: 2


## Performance analysis

![Performance of Remove operations](performance_remove.png)